# Final Tutorial: Analyzing IMDB Movie Dataset

install packages

In [20]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

Download and parse the csv into a pandas data frame. Only considering movies made in the USA for our analysis.

In [160]:
movies_table = pd.read_csv("movie_metadata.csv")

# replace na values with 0
movies_table["gross"].fillna(0, inplace=True)
movies_table["budget"].fillna(0, inplace=True)
movies_table["title_year"].fillna(0, inplace=True)
movies_table["country"].fillna("NaN", inplace=True)

# only consider movies made in the USA. Drop all other rows
movies_table.drop(movies_table[-(movies_table["country"].str.contains("USA"))].index, inplace=True)

movies_table.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0


scrape the annual average cpi values for every year from 1912-2016 exclusively

In [161]:
url = "http://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/"

r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, 'html.parser')

table = soup.find('table')
rows = table.tbody.findAll('tr');

years = []
cpis = []

for row in rows:
    year = row.findAll('td')[0].get_text()
    if year.isdigit() and int(year) < 2017:
        years.append(int(year))
        cpis.append(float(row.findAll('td')[13].get_text()))

cpi_table = pd.DataFrame({
    "year": years,
    "avg_annual_cpi": cpis
})

cpi_table.head()

,avg_annual_cpi,year
0,9.9,1913
1,10.0,1914
2,10.1,1915
3,10.9,1916
4,12.8,1917


Transform the budget and gross for each movie into real 2016 dollar terms. Drop the nominal value columns, as those won't contribute to our analysis.

In [162]:
real_domestic_gross = []
real_budget_values = []
cpi_2016 = float(cpi_table[cpi_table['year'] == 2016]['avg_annual_cpi'])

# function that translates nominal dollars into real dollars in 2016 using CPI 
def get_real_value(nominal_amt, old_cpi, cpi_2016):
    real_value = (nominal_amt * cpi_2016) / old_cpi
    return real_value

# drop all rows in movies_table with a budget, gross, or year of 0, as those rows won't contribute to our analysis
movies_table.drop(movies_table[(movies_table["budget"] == 0) | (movies_table["gross"] == 0) | 
                                (movies_table["title_year"] == 0)].index, inplace=True)


# must transform gross and budget values into real 2016 dollar terms
for index, row in movies_table.iterrows():
    gross = row['gross']
    budget = row['budget']
    year = row['title_year']
    cpi = float(cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi'])
    
    real_gross = get_real_value(gross, cpi, cpi_2016)
    real_budget = get_real_value(budget, cpi, cpi_2016)
    real_domestic_gross.append(real_gross)
    real_budget_values.append(real_budget)

movies_table["real_domestic_gross"] = real_domestic_gross
movies_table["real_budget"] = real_budget_values

# drop the gross and budget cols because we don't care about nominal values
movies_table.drop(labels='gross', axis=1, inplace=True)
movies_table.drop(labels='budget', axis=1, inplace=True)

movies_table.head()   

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,genres,actor_1_name,...,language,country,content_rating,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,real_domestic_gross,real_budget
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,...,English,USA,PG-13,2009.0,936.0,7.9,1.78,33000,8.507937e+08,2.651368e+08
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,Action|Adventure|Fantasy,Johnny Depp,...,English,USA,PG-13,2007.0,5000.0,7.1,2.35,0,3.582208e+08,3.473329e+08
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,Action|Thriller,Tom Hardy,...,English,USA,PG-13,2012.0,23000.0,8.5,2.35,164000,4.684551e+08,2.613385e+08
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,Action|Adventure|Sci-Fi,Daryl Sabara,...,English,USA,PG-13,2012.0,632.0,6.6,2.35,24000,7.637218e+07,2.756598e+08
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,Action|Adventure|Romance,J.K. Simmons,...,English,USA,PG-13,2007.0,11000.0,6.2,2.35,0,3.896268e+08,2.987063e+08


Let's calculate the absolute profit and ROI for each movie

In [163]:
profits = []
roi_vals = []

for index, row in movies_table.iterrows():
    profit = row['real_domestic_gross'] - row['real_budget']
    budget = row['real_budget']
    num = profit - budget
    den = budget
    # convert roi to percentage
    roi = (num / den) * 100
    
    profits.append(profit)
    roi_vals.append(roi)
    
movies_table['profit'] = profits
movies_table['roi'] = roi_vals

movies_table.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,genres,actor_1_name,...,content_rating,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,real_domestic_gross,real_budget,profit,roi
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,...,PG-13,2009.0,936.0,7.9,1.78,33000,8.507937e+08,2.651368e+08,5.856569e+08,120.888543
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,Action|Adventure|Fantasy,Johnny Depp,...,PG-13,2007.0,5000.0,7.1,2.35,0,3.582208e+08,3.473329e+08,1.088790e+07,-96.865283
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,Action|Thriller,Tom Hardy,...,PG-13,2012.0,23000.0,8.5,2.35,164000,4.684551e+08,2.613385e+08,2.071167e+08,-20.747743
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,Action|Adventure|Sci-Fi,Daryl Sabara,...,PG-13,2012.0,632.0,6.6,2.35,24000,7.637218e+07,2.756598e+08,-1.992877e+08,-172.294775
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,Action|Adventure|Romance,J.K. Simmons,...,PG-13,2007.0,11000.0,6.2,2.35,0,3.896268e+08,2.987063e+08,9.092051e+07,-69.561898


Let's group the movies by greatest Return on Investment. 

In [164]:
movies_by_roi = movies_table.sort_values('roi', ascending=False)

for index, row in movies_by_roi.head().iterrows():
    print(row["movie_title"], row["roi"])

Paranormal Activity  719248.5533333333
Tarnation  271366.05504587153
The Blair Witch Project  234016.85666666666
The Brothers McMullen  40786.40000000001
The Texas Chain Saw Massacre  36742.72853517215


Very interesting to see how out of the top 5 movies with greatest ROI, 3/5 films are horror.

Let's group the movies by greatest absolute profit.

In [165]:
movies_by_profit = movies_table.sort_values('profit', ascending=False)

for index, row in movies_by_profit.head().iterrows():
    print(row["movie_title"], row["profit"])

Gone with the Wind  3361449602.0105033
Snow White and the Seven Dwarfs  3048847005.4440975
Star Wars: Episode IV - A New Hope  1781975398.5091584
Pinocchio  1400612278.5714285
Fantasia  1270665631.4285712


Let's group the movies by greatest real domestic gross at the box office.

In [166]:
movies_by_gross = movies_table.sort_values('real_domestic_gross', ascending=False)

for index, row in movies_by_gross.head().iterrows():
    print(row["movie_title"], row["real_domestic_gross"])

Gone with the Wind  3430119230.7155395
Snow White and the Seven Dwarfs  3082181310.999653
Star Wars: Episode IV - A New Hope  1825541025.5718646
Pinocchio  1445185007.142857
Fantasia  1309752485.7142856


Let's group the movies by greatest imdb scores.

In [167]:
movies_by_score = movies_table.sort_values('imdb_score', ascending=False)

for index, row in movies_by_score.head().iterrows():
    print(row["movie_title"], row["imdb_score"], row["real_domestic_gross"])

The Shawshank Redemption  9.3 45898454.45535088
The Godfather  9.2 774119909.896268
The Dark Knight  9.0 594509077.2187428
The Godfather: Part II  9.0 278953369.168357
Pulp Fiction  8.9 174790523.0094467
